We have a dataset (DCC) with a shortage of certain labels. We want to generate 
new samples synthetically using GPT-4. We will use the following approach:
1. We take the existing samples for each document type and present these to GPT-4
2. we ask to generate new sentences like it, where the token labels are provided in the BIO format

We care specifically about the following labels:
* Experiencer: Other
* Historical: Hypothetical

The task of the GPT model is to generate new sentences that are similar to the input sentences but with variations of the medical concepts. 

For instance: 



In [95]:
import os, sys, re
import json, dotenv

import openai
import asyncio
from openai import AsyncOpenAI, OpenAI
from tqdm import tqdm

dotenv.load_dotenv()

True

In [2]:
openai.api_key = os.getenv("OPENAI_KEY")

In [3]:
DCC = json.load(open('../data/emc-dcc_ann.json'))

In [4]:
docs = DCC['projects'][0]['documents']

In [11]:
relevant_docs_hypothetical = []
for i, doc in enumerate(docs):
    for concept in doc['annotations']:
        if (concept['meta_anns']['Temporality']['value']=='hypothetical'):
            relevant_docs_hypothetical.append(doc)
            break
        
relevant_docs_experiencer = []
for i, doc in enumerate(docs):
    for concept in doc['annotations']:
        if (concept['meta_anns']['Experiencer']['value']=='other'):
            relevant_docs_experiencer.append(doc)
            break


In [92]:
relevant_docs_experiencer[12]['text'][0:36]

'#Name# liep op 12.12.1995 een trauma'

In [90]:
[(d['start'],d['end']) for d in relevant_docs_experiencer[12]['annotations'] 
 if d['meta_anns']['Experiencer']['value']=='other']

[(30, 36)]

In [6]:
OAI_ASYNC_CLIENT = AsyncOpenAI(api_key=os.getenv("OPENAI_KEY"), max_retries=2)
OAI_CLIENT = OpenAI(api_key=os.getenv("OPENAI_KEY"), max_retries=2)

In [102]:
SYSTEM_PROMPT_HYPOTHETICAL = """
    Je bent een kritische assistent die mij helpt om nieuwe zinnen te bedenken.
    Deze zinnen moeten voldoen aan de volgende eisen:
    - ze moeten semantisch correct zijn en vergelijkbaar zijn met de voorbeeldzinnen die ik je geef.
    - de voorbeeldzinnen worden voorafgegaan door de term VOORBEELDZIN
    - in de voorbeeldzin worden 1 of meer concepten benoemd die hypothethisch zijn, het is belangrijk
    dat deze concepten in de nieuwe zin ook hypothetisch zijn, het mogen ook andere concepten zijn. 
    Een voorbeeld van een hypothetische concept = 'een voorafgaand trauma kan niet worden herinnerd', waarin 'trauma' het concept is.
    Een ander voorbeeld = 'ter uitsluiting van epifysaire dysplasie' waarin 'epifysaire dysplasie' het concept is.
    - het domein is medisch dus gebruik medische concepten.
    - probeer de medische concepten te varieren, dus gebruik niet steeds dezelfde concepten.
    - geef als antwoord alleen de nieuw gegenereerde zinnen, voorafgaand met de term NIEUWE_ZIN
    - in de NIEUWE_ZIN, plaats de concepten die hypothetisch zijn tussen verticale streepjes, dus '|', 
    dus bijvoorbeeld: 'ter uitsluiting van |epifysaire dysplasie|'
"
"""

SYSTEM_PROMPT_EXPERIENCER = """
    Je bent een kritische assistent die mij helpt om nieuwe zinnen te bedenken.
    Deze zinnen moeten voldoen aan de volgende eisen:
    - ze moeten semantisch correct zijn en vergelijkbaar zijn met de voorbeeldzinnen die ik je geef.
    - de voorbeeldzinnen worden voorafgegaan door de term VOORBEELDZIN
    - in de voorbeeldzin worden 1 of meer concepten benoemd die verwijzen naar een persoon anders dan de patient, het is belangrijk
    dat deze concepten in de nieuwe zin ook verwijzen naar iemand anders dan de patient (zoals een familielid), 
    het mogen ook andere medische  concepten zijn. 
    Een voorbeeld van een concept wat verwijst naar een ander persoon dan de patient =
    'Een zusje van #Name# is elders operatief behandeld in verband met recidiverende patella luxaties', waarin 'luxaties' het concept is, en er 
    wordt verwezen naar de zus van de patient.
    Een ander voorbeeld = 'ter uitsluiting van epifysaire dysplasie' waarin 'epifysaire dysplasie' het concept is.
    - het domein is medisch dus gebruik medische concepten.
    - probeer de medische concepten te varieren, dus gebruik niet steeds dezelfde concepten.
    - geef als antwoord alleen de nieuw gegenereerde zinnen, voorafgaand met de term NIEUWE_ZIN
    - in de NIEUWE_ZIN, plaats de concepten die verwijzen naar een ander persoon dan de patient tussen tussen verticale streepjes |, 
    dus bijvoorbeeld: 'Een zusje van #Name# is elders operatief behandeld in verband met recidiverende patella |luxaties|'
"""

In [103]:
def get_chat_res(USER_TEXT='Good day', 
                 SYSTEM_PROMPT=SYSTEM_PROMPT_HYPOTHETICAL, 
                 MODEL="gpt-4"):
    return OAI_CLIENT.chat.completions.create(
            model=MODEL,
            n = 10,
            messages=[
                        {"role": "system",
                        "content": SYSTEM_PROMPT
                        },
                        {"role": "user", 
                        "content": USER_TEXT
                        }],
            stream=False,
        )

In [47]:
#re_extract = re.compile(r'NIEUWE_ZIN\:(.*)')
nieuwe_zinnen_hypothetisch = []
for i, doc in tqdm(enumerate(relevant_docs_hypothetical)):
    res = get_chat_res(SYSTEM_PROMPT=SYSTEM_PROMPT_HYPOTHETICAL, 
                       USER_TEXT="Parafraseer de VOORBEELDZIN: " + doc['text'])
    # extract the part after "NIEUWE_ZIN"
    for _res in res.choices:
        txt = _res.message.content
        nieuwe_zinnen_hypothetisch.append(txt[txt.find('NIEUWE_ZIN:')+12:].strip())

In [104]:
#re_extract = re.compile(r'NIEUWE_ZIN\:(.*)')
nieuwe_zinnen_experiencer = []
for i, doc in tqdm(enumerate(relevant_docs_experiencer)):
    res = get_chat_res(SYSTEM_PROMPT=SYSTEM_PROMPT_EXPERIENCER, 
                       USER_TEXT="Parafraseer de VOORBEELDZIN: " + doc['text'])
    # extract the part after "NIEUWE_ZIN"
    for _res in res.choices:
        txt = _res.message.content
        nieuwe_zinnen_experiencer.append(txt[txt.find('NIEUWE_ZIN:')+12:].strip())
    break

0it [00:09, ?it/s]


In [105]:
print(nieuwe_zinnen_experiencer[2], sep='\n')

Gezien het prominent aanwezige |spasme| van de flexoren, lijkt een aanvang met conservatieve therapie zoals fysiotherapie, spalken, en pijnbestrijding, geen overbodige luxe.
NIEUWE_ZIN: De patiënt ervaart naast zijn fysieke symptomen ook psychologische stress in verband met eerdere |trauma's| gerelateerd aan zijn ouders, waarbij ondersteuning wordt geboden via het Boumanshuis.
NIEUWE_ZIN: De |flexiecontractuur| lijkt meer een resultaat te zijn van een spiertrekking van de flexoren dan van een andere oorzaak.


* <English> BioScope; [HF](https://huggingface.co/datasets/bigbio/bioscope), [src](https://rgai.inf.u-szeged.hu/downloads)
* <English> [Genia](http://www.geniaproject.org/genia-corpus/term-corpus)
